In [ ]:
import numpy as np
from stonesoup.types.state import GaussianState

def retrodict(current_state, prior_state, transition_model):
    delta_t = current_state.timestamp - prior_state.timestamp
    F = transition_model.matrix(timedelta = delta_t)
    D = transition_model.covar(timedelta = delta_t)

    x_ll = prior_state.mean     # Vorzustand (der verbessert wird)
    P_ll = prior_state.covar

    x_l1k = current_state.mean  # momentaner Zustand
    P_l1k = current_state.covar

    x_l1l = F @ x_ll    # predizierter Zustand
    P_l1l = F @ P_ll @ np.transpose(F) + D

    W_l1l = P_ll @ np.transpose(F) @ np.linalg.pinv(P_l1l)   # Gewichtsmatrix

    x_lk = x_ll + W_l1l @ (x_l1k - x_l1l)   # verbesserter Zustand
    P_lk = P_ll + W_l1l @ (P_l1k - P_l1l) @ np.transpose(W_l1l)

    return GaussianState(x_lk, P_lk, timestamp=prior_state.timestamp)

